In [1]:
import pandas as pd
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
%matplotlib inline

Using TensorFlow backend.


## 1. Data Preparation

In [17]:
data = pd.read_excel('../data/Testcase_dataset-IBPEM.xlsx')
# Create timestamp from two columns
data['timestamp'] = pd.to_datetime({
    'year': data['Date'].str.split('-').str.get(0), 
    'month': data['Date'].str.split('-').str.get(1), 
    'day': data['Date'].str.split('-').str.get(2), 
    'hour':data['Time'].str.slice(0, 2), 
    'minute': data['Time'].str.slice(2,4)})

data = data.drop(columns=['Date', 'Time'])
data = data.set_index('timestamp')

In [28]:
# Let's take only the CWRT_syth first
series1 = data['CWRT_syth'].dropna()
n_input = 12
n_output = 6
cwrt_data = series1.to_list()

encoder_data, decoder_data, target_data = list(), list(), list()
for i in range(0,len(cwrt_data)-n_input-n_output):
    encoder_input = cwrt_data[i:i+n_input-1]
    target_input = cwrt_data[i+n_input:i+n_input+n_output-1]
    decoder_input = [-1.0] + target_input[:-1]
    
    encoder_data.append(encoder_input)
    decoder_data.append(decoder_input)
    target_data.append(target_input)

encoder_data = array(encoder_data)
decoder_data = array(decoder_data)
target_data = array(target_data)

In [29]:
decoder_data

array([[-1.        , 31.3159996 , 31.34000015, 31.35999985, 31.38999939],
       [-1.        , 31.34000015, 31.35999985, 31.38999939, 31.38999939],
       [-1.        , 31.35999985, 31.38999939, 31.38999939, 31.40599976],
       ...,
       [-1.        , 37.23600082, 36.67799988, 37.02399979, 36.44599991],
       [-1.        , 36.67799988, 37.02399979, 36.44599991, 36.11999893],
       [-1.        , 37.02399979, 36.44599991, 36.11999893, 35.90200119]])

## 2. LSTM encoder-decoder model

In [30]:
# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
    # define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    # define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    # return all models
    return model, encoder_model, decoder_model

In [31]:
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = array([-1.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat
    return array(output)

In [9]:
# configure problem
n_features = 1
n_input = 12
n_steps_out = 6
# define model
train, infenc, infdec = define_models(1, 1, 128)

train.compile(optimizer='adam', loss='crossentropy', metrics=['acc'])

Epoch 1/300
1/1 [==============================] - 51s 51s/step - loss: 1021525632.0000
Epoch 2/300
1/1 [==============================] - 36s 36s/step - loss: nan
Epoch 3/300
1/1 [==============================] - 36s 36s/step - loss: nan
Epoch 4/300
1/1 [==============================] - 34s 34s/step - loss: nan
Epoch 5/300
1/1 [==============================] - 33s 33s/step - loss: nan
Epoch 6/300
1/1 [==============================] - 36s 36s/step - loss: nan
Epoch 7/300
1/1 [==============================] - 36s 36s/step - loss: nan
Epoch 8/300


KeyboardInterrupt: 